<a 
href="https://colab.research.google.com/github/adilo231/Python-for-machine-learning/blob/main/Chap -4- Machine learning applications/3- Compose make column.ipynb" 
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imputers

In [12]:
from sklearn.impute import SimpleImputer
import numpy as np

In [23]:
X= np.array([[10,3],
[3,5],
[5,6],
[np.nan,3]])
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')
imputer2 = SimpleImputer(missing_values=np.nan,strategy='median')
imputer3 = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer4 = SimpleImputer(missing_values=np.nan,strategy='constant', fill_value=99)

print(imputer.fit_transform(X))
print(imputer2.fit_transform(X))
print(imputer3.fit_transform(X))
print(imputer4.fit_transform(X))

[[10.  3.]
 [ 3.  5.]
 [ 5.  6.]
 [ 6.  3.]]
[[10.  3.]
 [ 3.  5.]
 [ 5.  6.]
 [ 5.  3.]]
[[10.  3.]
 [ 3.  5.]
 [ 5.  6.]
 [ 3.  3.]]
[[10.  3.]
 [ 3.  5.]
 [ 5.  6.]
 [99.  3.]]


## Make Column transformer

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split,GridSearchCV

In [11]:
titanic = sns.load_dataset('titanic')
titanic = titanic.drop([ 'class'	
                        	, 'embarked',  'alive'	], axis=1)
titanic.dropna(axis=0, inplace=True)
y = titanic['survived']
X = titanic.drop('survived', axis=1)
numerical_features = ['pclass', 'age', 'fare','sibsp']
categorical_features = ['sex', 'deck', 'alone','who','adult_male','embark_town']

X_train, X_test, y_train, y_test = train_test_split(X, y)
print(y.shape)
print(X.shape)
titanic.head()

(182,)
(182, 11)


,survived,pclass,sex,age,sibsp,parch,fare,who,adult_male,deck,embark_town,alone
1,1,1,female,38.0,1,0,71.2833,woman,False,C,Cherbourg,False
3,1,1,female,35.0,1,0,53.1000,woman,False,C,Southampton,False
6,0,1,male,54.0,0,0,51.8625,man,True,E,Southampton,True
10,1,3,female,4.0,1,1,16.7000,child,False,G,Southampton,False
11,1,1,female,58.0,0,0,26.5500,woman,False,C,Southampton,True


In [8]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier 
from sklearn.neighbors import KNeighborsClassifier




In [58]:


transformer = make_column_transformer((StandardScaler(), ['age', 'fare']))
transformer.fit(X)

model = make_pipeline(transformer, SGDClassifier())
model.fit(X, y)




Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'fare'])])),
                ('sgdclassifier', SGDClassifier())])

In [100]:
numerical_pipline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipline = make_pipeline(SimpleImputer(
    strategy='most_frequent'), OneHotEncoder())
preprocessor = make_column_transformer((numerical_pipline, numerical_features),
                                      (categorical_pipline, categorical_features))


In [101]:
model = make_pipeline(preprocessor, SGDClassifier())


In [106]:
params = {
   
    'sgdclassifier__penalty':['l1', 'l2']
}
grid = GridSearchCV(model, param_grid=params, cv=10)
grid.fit(X_train, y_train)
print(grid.score(X_test,y_test))

0.5652173913043478


In [108]:
numerical_pipline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipline = make_pipeline(SimpleImputer(
    strategy='most_frequent'), OneHotEncoder())
    
preprocessor = make_column_transformer((numerical_pipline, numerical_features),
                                      (categorical_pipline, categorical_features))
model = make_pipeline(preprocessor, KNeighborsClassifier())
params = {
       'kneighborsclassifier__n_neighbors': np.arange(1, 20),
    'kneighborsclassifier__metric': ['minkowski','euclidean', 'manhattan']
}
grid = GridSearchCV(model, param_grid=params, cv=5)
grid.fit(X_train, y_train)
print(grid.score(X_test,y_test))

0.717391304347826


## Make column selector

In [6]:
from sklearn.compose import make_column_selector

In [9]:
numerical_features =make_column_selector(dtype_include=np.number)
categorical_features = make_column_selector(dtype_exclude=np.number)

numerical_pipline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipline = make_pipeline(SimpleImputer(
    strategy='most_frequent'), OneHotEncoder())
    
preprocessor = make_column_transformer((numerical_pipline, numerical_features),
                                      (categorical_pipline, categorical_features))
model = make_pipeline(preprocessor, KNeighborsClassifier())
params = {
       'kneighborsclassifier__n_neighbors': np.arange(1, 20),
    'kneighborsclassifier__metric': ['minkowski','euclidean', 'manhattan']
}
grid = GridSearchCV(model, param_grid=params, cv=5)
grid.fit(X_train, y_train)
print(grid.score(X_test,y_test))

0.8043478260869565
